## Methodische & Digitale Kompetenz (DIKO) - Modularbeit Eigenleistung - SoSe 2024 - Maximilian Fiebig
## Experiment Temperature

#### Installation von OpenAI-Bibliothek (https://pypi.org/project/openai/)

In [17]:
pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


#### Erstellung der Azure OpenAI-Instanz (https://learn.microsoft.com/de-de/azure/ai-services/openai/how-to/switching-endpoints#api-key)
- Der API KEY ist ähnlich wie ein Passwort
- Man speichert ihn in einer externen .env-Datei, damit niemand den Key im Code sehen kann
- Die API-Version ist die genaue Spezifikation/Anleitung, damit die Bibliothek genau weiß, wie das JSON formatiert werden muss. Sie ändert sich durch Updates, deswegen muss sie mit angegeben werden. Die Version 2024-02-01 ist die von MUCGPT genutze Version bei Azure. (https://learn.microsoft.com/en-gb/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release)

In [18]:
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv() # Laden der externen .env-Datei, in der API-Key und Azure Endpoint gespeichert sind.

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

Chat Completions API-Aufruf: https://platform.openai.com/docs/guides/text-generation/chat-completions-api

- Temperature: Im Prompting bestimmt die Temperatur die Gewichtung von Kreativität in den Antworten der KI. Sie kann auch die Konsistenz und Vorhersehbarkeit der KI-Antworten beeinflussen. Eine niedrigere Temperatur führt dazu, dass die KI eher Wörter wählt, die wahrscheinlicher sind, und die Antworten beim gleichen Prompt sich ähnlich sind. Näheres dazu finden Sie am Ende des Codes.

In [19]:
response = client.chat.completions.create(
  model="chat", # 'Chat' ist das zugehörige Deployment bei Azure, dahinter liegt gpt-3.5-turbo.
  messages=[
    {"role": "system", "content": "Du bist ein Experte für Baustoffkunde."}, # Anweisung an das Sprachmodell, optional
    {"role": "user", "content": "Was sind die Bestandteile von Beton?"} # Frage von Nutzer an das Sprachmodell
  ], 
  temperature=1 # Steuerung der Kreativität
)

In [20]:
print(response.choices[0].message.content) # Ausgabe der Antwort des Chatmodells

Die Bestandteile von Beton sind Zement, Sand, Kies oder Schotter (Gesteinskörnung) und Wasser. Zusätzlich können noch Zusatzstoffe wie z.B. Zuschlagstoffe, Superplastifizierer, Luftporenbildner oder Farbstoffe hinzugefügt werden, um bestimmte Eigenschaften des Betons zu verbessern oder zu verändern.


#### Importieren der .txt-Datei mit Fragen

In [21]:
with open("Fragen.txt") as datei:
    fragen = datei.readlines() # In der txt.-Datei sollen alle Zeilen gelesen werden.

print(fragen)

['Wie viel wiegt 1m³ Beton?\n', 'Gibt es klimaneutrale Baustoffe?\n', 'Welche Eigenschaften hat Asphalt?\n', 'Wie wird ein Ziegelstein hergestellt?\n', 'Kann man eine Straße aus Beton bauen?\n', 'Aus welchem Baustoff bestehen Brücken?\n', 'Wie sieht die Rezyklierbarkeit von Beton aus?\n', 'Ist Stahl ein guter Baustoff für U-Bahngleise?\n', 'Ist es möglich, ein Haus aus dem Baustoff Holz zu bauen?\n', 'Welches Material wird für den Einsatz in Hochbauten eingesetzt?']


In [22]:
fragen = [line.rstrip() for line in fragen] # .rstrip entfernt Zeilenumbürche (/n), https://statistikguru.de/python/python-zeilenweise-lesen.html
print(fragen)

['Wie viel wiegt 1m³ Beton?', 'Gibt es klimaneutrale Baustoffe?', 'Welche Eigenschaften hat Asphalt?', 'Wie wird ein Ziegelstein hergestellt?', 'Kann man eine Straße aus Beton bauen?', 'Aus welchem Baustoff bestehen Brücken?', 'Wie sieht die Rezyklierbarkeit von Beton aus?', 'Ist Stahl ein guter Baustoff für U-Bahngleise?', 'Ist es möglich, ein Haus aus dem Baustoff Holz zu bauen?', 'Welches Material wird für den Einsatz in Hochbauten eingesetzt?']


#### Erstellen Funktion und Liste zum auslesen der Fragen und Antworten

In [23]:
def frage_gpt(text, temp=0): #Definieren der Funktion frage_gpt, und Übergabe der Eingabewerte. 'text' fungiert als Platzhalter, Temperature wird auf 0 festgelegt.
    response = client.chat.completions.create(
        model="chat",
        messages=[
            {"role": "system", "content": "Du bist ein Experte für Bauingenieurwesen. Antworte kurz & prägnant auf Fragen."}, # Aneweisung an das Sprachmodell
            {"role": "user", "content": text}
        ], 
        temperature=temp
    )
    return response.choices[0].message.content

In [24]:
import os
os.remove('Ausgabe_Temp.txt') # Öffnen der Datei Ausgabe_Temp.txt, um die Fragen und Antworten in dieser Datei auszugeben.

antworten = [] #Erstellen einer Liste für die Ausgabe der Antworten.

file = open('Ausgabe_Temp.txt', 'a')
for frage in fragen: # Schleife, um alle Fragen aus der Liste abzuarbeiten und die entsprechende Antwort des Chatmodells mit abzufragen.
    antwort00 = frage_gpt(frage) # Festlegen der verschiedenen Temperatures des Chatmodells
    antwort07 = frage_gpt(frage, temp=0.7)
    antwort15 = frage_gpt(frage, temp=1.5)
    output_text = "Frage: {}\nAntwort 0.0: {}\nAntwort 0.7: {}\nAntwort 1.5: {}\n\n".format(frage, antwort00, antwort07, antwort15) # Formatierung der Antwortausgabe.
    file.write(output_text) # Übergabe der Antwort an die .txt-Ausgabedatei.
    print(output_text)

file.close()

Frage: Wie viel wiegt 1m³ Beton?
Antwort 0.0: 1m³ Beton wiegt etwa 2400 kg bis 2500 kg, je nach Art des Betons und seiner Zusammensetzung.
Antwort 0.7: Die Dichte von Beton variiert je nach Art des Betons, aber in der Regel wiegt 1m³ Beton etwa 2400 kg bis 2500 kg.
Antwort 1.5: 1m³ Beton wiegt in der Regel etwa 2.400 bis 2.500 kg.<IActionResult>


Frage: Gibt es klimaneutrale Baustoffe?
Antwort 0.0: Ja, es gibt klimaneutrale Baustoffe wie zum Beispiel Holz, Lehm oder Stroh. Diese Materialien binden während ihres Wachstums CO2 und setzen es nicht bei der Herstellung frei. Zudem können sie oft recycelt oder wiederverwendet werden.
Antwort 0.7: Ja, es gibt klimaneutrale Baustoffe wie zum Beispiel Holz, Kork oder Lehm. Diese Materialien binden während ihres Wachstums oder bei der Herstellung mehr CO2, als bei ihrer Verwendung freigesetzt wird.
Antwort 1.5: Es gibt Baustoffe, die als klimaneutral gelten, da sie bei ihrer Produktion kein C02 ausstoßen oder recycelt werden können. Dazu gehöre

#### Erkenntnis: Grundsätzlich bestimmt die Temperatur die Gewichtung von Kreativität in den Antworten der KI. Die Temperatur bietet aber mehr als nur eine Möglichkeit, die Kreativität zu steuern. Sie kann auch die Konsistenz und Vorhersehbarkeit der KI-Antworten beeinflussen. Eine niedrigere Temperatur führt dazu, dass die KI eher Wörter wählt, die wahrscheinlicher sind, und die Antworten beim gleichen Prompt sich ähnlich sind (https://www.german.ai/wissen/temperatur). Es gibt drei verschiedene Level der Temperature. Niedrige Temperatur (bei ChatGPT 0.1 - 0.4): Hier erhält man präzise, neutrale Antworten. Mittlere Temperatur (bei ChatGPT 0.5 - 0.7): Diese Einstellung hält die Balance zwischen Kreativität und Konsistenz. 0.7 soll die Standardtemperatur von ChatGPT sein. Hohe Temperatur (bei ChatGPT 0.8 - 1.0): Mit dieser Einstellung generiert die AI überraschende, unvorhersehbare und kreative Antworten. Manchmal findest man gute Antworten, und manchmal schießt die AI am Ziel vorbei. Hierbei können vorallem Schreibfehler und sinnlose Wortaneinanderreihungen das Ergbenis sein. Diese Phänomene lassen sich anhand der oben gezeigten Beispiele optimal nachvollziehen.